This notebook demonstrates ingesting ENCODE bigWig into tileDB dense array with S3 backend.

# Why use tileDB

# What is bigWig

# How to configure S3 backend

# Create tileDB array

# Open bigWig with pyBigWig

# Write data to tileDB

# Query data